In [1]:
!nvidia-smi

Wed Mar 24 00:52:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install all needed packages
!pip install pytorch-lightning
!pip install git+https://github.com/kornia/kornia
!pip uninstall -y tensorflow  # Latest tensorflow breaks logging in pytorch-lightning
!pip install tensorboard

     |████████████████████████████████| 829kB 13.9MB/s 
     |████████████████████████████████| 829kB 27.8MB/s 
     |████████████████████████████████| 276kB 56.5MB/s 
     |████████████████████████████████| 112kB 54.4MB/s 
     |████████████████████████████████| 1.3MB 54.5MB/s 
     |████████████████████████████████| 143kB 58.2MB/s 
     |████████████████████████████████| 296kB 51.3MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=67c38e7529741b386163316cc5ac3a48552cc7c9afa5f8fdfbb11897f2ec2b92
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=820102d370fa892cdf0dd5c2262b14588181e2fff11d181b93615ef14f2b4f66
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built future PyYAML
  Found existing installation: future 0.16.0
  

In [3]:
import os
import pathlib

In [4]:
# WARNING: Put here path to the directory where CUFEDS's input.zip, ref.zip and CUFED5.zip are located. Instruction of how to get all files are in README
gd_data_dirpath = pathlib.Path('drive/MyDrive/skoltech/data') 

input_zippath = gd_data_dirpath / 'input.zip'
ref_zippath = gd_data_dirpath / 'ref.zip'
gt_zippath = gd_data_dirpath / 'CUFED5.zip'

data_dirpath = pathlib.Path('CUFED')
train_data_dirpath = pathlib.Path('CUFED') / 'train'
test_data_dirpath = pathlib.Path('CUFED') / 'test'

In [5]:
# Prepare the data (unzipping and stuff...)

!mkdir -p CUFED/train
!mkdir -p CUFED/test

!unzip -uq $input_zippath -d $train_data_dirpath
!unzip -uq $ref_zippath -d $train_data_dirpath
!unzip -uq $gt_zippath -d $test_data_dirpath

In [10]:
# Close our repo

!rm -rf train-suite
!git clone https://github.com/theleokul/train-suite.git

Cloning into 'train-suite'...
remote: Enumerating objects: 584, done.
remote: Counting objects: 100% (584/584), done.
remote: Compressing objects: 100% (401/401), done.
remote: Total 584 (delta 351), reused 394 (delta 161), pack-reused 0
Receiving objects: 100% (584/584), 5.16 MiB | 14.72 MiB/s, done.
Resolving deltas: 100% (351/351), done.


In [24]:
# Training baseline (for 2 epochs with reconstruction loss only)
# In order to reproduce different experiments, change the config (all are available in our repo)
!python train-suite/main.py --config train-suite/config/train/exp_8/ttsr_warmup_colab.yml -g 0 \
    -L 'logs'  # Path for logs and checkpoints

Running experiment 8...
GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name  | Type | Params
-------------------------------
0 | model | TTSR | 7.3 M 
-------------------------------
6.8 M     Trainable params
555 K     Non-trainable params
7.3 M     Total params
29.263    Total estimated model params size (MB)
Epoch 0:   3% 40/1445 [00:35<21:02,  1.11it/s, loss=0.473, v_num=4, a_ssim=0.293, a_psnr=11.10]


In [26]:
# Fine-tune the baseline
!python train-suite/main.py --config train-suite/config/train/exp_8/ttsr_colab.yml \
    -g 0 \
    -b 'drive/MyDrive/skoltech/exp_8/lightning_logs/version_0/checkpoints/epoch=1-a_psnr=25.2712-a_ssim=0.7421.ckpt' \
    -L 'logs'  # Path for logs and checkpoints

Running experiment 8...
Loaded: drive/MyDrive/skoltech/exp_8/lightning_logs/version_0/checkpoints/epoch=1-a_psnr=25.2712-a_ssim=0.7421.ckpt
Running experiment 8...
GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name          | Type          | Params
------------------------------------------------
0 | model         | TTSR          | 7.3 M 
1 | discriminator | Discriminator | 17.8 M
2 | vgg19         | Vgg19         | 12.9 M
------------------------------------------------
24.6 M    Trainable params
13.5 M    Non-trainable params
38.1 M    Total params
152.329   Total estimated model params size (MB)
Epoch 0:   4% 60/1445 [01:16<29:32,  1.28s/it, loss=0.707, v_num=5, a_ssim=0.705, a_psnr=23.80]


In [21]:
# Get predictions on validation dataset
!python train-suite/main.py --config train-suite/config/test/exp_9/ttsr_colab.yml -g 0 \
    --modes predict \
    -m 'drive/MyDrive/skoltech/exp_9/lightning_logs/version_1/checkpoints/epoch=49-a_psnr=22.3143-a_ssim=0.5979.ckpt' \
    -O 'outputs'  # Output directory for predictions

Loaded: drive/MyDrive/skoltech/exp_9/lightning_logs/version_1/checkpoints/epoch=49-a_psnr=22.3143-a_ssim=0.5979.ckpt
  8% 10/126 [00:06<01:11,  1.62it/s]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/PIL/ImageFile.py", line 496, in _save
    fh = fp.fileno()
AttributeError: '_idat' object has no attribute 'fileno'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "train-suite/main.py", line 184, in <module>
    main()
  File "train-suite/main.py", line 178, in main
    lit_model.predict_step(batch, i, output_dirpath, device)
  File "/content/train-suite/composers/ttsr_lit_composer.py", line 82, in predict_step
    imageio.imsave(os.path.join(output_dirpath, f'{batch_idx}_ref.png'), ref)
  File "/usr/local/lib/python3.7/dist-packages/imageio/core/functions.py", line 260, in imwrite
    writer.append_data(im)
  File "/usr/local/lib/python3.7/dist-packages/imageio/core/format.py", line 500, i

In [22]:
# Estimate metrics
!python train-suite/main.py --config train-suite/config/test/exp_9/ttsr_colab.yml -g 0 \
    --modes test \
    -m 'drive/MyDrive/skoltech/exp_9/lightning_logs/version_1/checkpoints/epoch=49-a_psnr=22.3143-a_ssim=0.5979.ckpt'

Loaded: drive/MyDrive/skoltech/exp_9/lightning_logs/version_1/checkpoints/epoch=49-a_psnr=22.3143-a_ssim=0.5979.ckpt
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Testing: 100% 126/126 [00:38<00:00,  3.26it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'a_psnr': 22.3143253326416, 'a_ssim': 0.5979177310099265}
--------------------------------------------------------------------------------
